<a href="https://colab.research.google.com/github/MarianVelasquez/Sentiment-Analysis-and-Text-Mining/blob/main/Assigment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import pickle
import pathlib
import numpy as np
import pandas as pd
import streamlit as st

In [5]:

MODEL_PATH = pathlib.Path("model.pkl")
FEATURES = ["RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe"]

In [6]:

st.set_page_config(page_title="Glass Type Predictor", page_icon="🧪", layout="centered")

2025-08-13 19:04:53.731 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [7]:
@st.cache_data
def load_data():
    # 1) prefer local CSV
    if pathlib.Path("glass.csv").exists():
        return pd.read_csv("glass.csv")
    # 2) else try URL
    url = os.environ.get("GLASS_CSV_URL", "").strip()
    if url:
        return pd.read_csv(url)
    # 3) otherwise empty df – user can still use the app with the pre-trained model
    return pd.DataFrame(columns=FEATURES + ["Type"])

@st.cache_resource
def load_model():
    if MODEL_PATH.exists():
        with open(MODEL_PATH, "rb") as f:
            return pickle.load(f)
    st.warning("No model.pkl found. Train it first (run python train.py).")
    return None

df = load_data()
model = load_model()

st.title("🧪 Glass Type Prediction")
st.markdown(
    "Enter chemical analysis values to predict the *glass Type* "
    "(as per UCI Glass Identification dataset)."
)

# Nice defaults from dataset (if available)
defaults = df[FEATURES].median().to_dict() if not df.empty else {
    "RI": 1.52, "Na": 13.0, "Mg": 3.6, "Al": 1.4, "Si": 72.0, "K": 0.6, "Ca": 8.7, "Ba": 0.0, "Fe": 0.0
}

with st.form("inputs"):
    c1, c2, c3 = st.columns(3)
    RI = c1.number_input("RI (Refractive Index)", value=float(defaults["RI"]), format="%.5f")
    Na = c2.number_input("Na", value=float(defaults["Na"]), format="%.3f")
    Mg = c3.number_input("Mg", value=float(defaults["Mg"]), format="%.3f")

    c4, c5, c6 = st.columns(3)
    Al = c4.number_input("Al", value=float(defaults["Al"]), format="%.3f")
    Si = c5.number_input("Si", value=float(defaults["Si"]), format="%.3f")
    K  = c6.number_input("K",  value=float(defaults["K"]),  format="%.3f")

    c7, c8, c9 = st.columns(3)
    Ca = c7.number_input("Ca", value=float(defaults["Ca"]), format="%.3f")
    Ba = c8.number_input("Ba", value=float(defaults["Ba"]), format="%.3f")
    Fe = c9.number_input("Fe", value=float(defaults["Fe"]), format="%.3f")

    submitted = st.form_submit_button("Predict")

if submitted:
    if model is None:
        st.error("Model not available. Please train first with python train.py.")
    else:
        X = pd.DataFrame([[RI, Na, Mg, Al, Si, K, Ca, Ba, Fe]], columns=FEATURES)
        pred = model.predict(X)[0]
        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(X).flatten()
            # Map class labels to probabilities
            classes = model.classes_
            proba_df = pd.DataFrame({"Type": classes, "Probability": proba}).sort_values("Type")
            st.subheader(f"Prediction: *Type {int(pred)}*")
            st.bar_chart(proba_df.set_index("Type"))
            with st.expander("Raw probabilities"):
                st.dataframe(proba_df.reset_index(drop=True))
        else:
            st.subheader(f"Prediction: *Type {int(pred)}*")

st.divider()
st.markdown("#### About the model")
st.markdown(
    "- Trained with *LogisticRegression* on the UCI Glass dataset.\n"
    "- Target: Type (1–7). Features: RI, Na, Mg, Al, Si, K, Ca, Ba, Fe.\n"
    "- No personal data is processed or stored. Inputs stay in your browser session."
)

2025-08-13 19:04:57.130 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-08-13 19:04:57.133 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-08-13 19:04:57.134 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 19:04:57.214 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-08-13 19:04:57.215 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 19:04:57.216 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 19:04:57.217 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 19:04:57.223 Thread 'MainThread': missin

DeltaGenerator(_form_data=FormData(form_id='inputs'))

In [3]:
%pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 114.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.6 MB/s eta 0:00:00
